# Chapter 1

This Jupyter Notebook implements key parts of Chapter 1 of the book "An Introduction to Tsetlin Machines". The complete chapter can be found here: https://www.researchgate.net/publication/354450962_An_Introduction_to_Tsetlin_Machines_Your_First_Tsetlin_Machine.

## Boolean Features

You first need observations that the Tsetlin machine can learn from. The following dataset contains three objects of type _Car_. They are characterized by five Boolean features: _Four Wheels_, _Transports People_, _Wings_, _Yellow_, and _Blue_. The first object in the dataset, for instance, has _Four Wheels_, _Transports People_, does not have _Wings_, is _Blue_, but not _Yellow_. 

In [1]:
cars = [
    {'Four Wheels':True, 'Transports People':True, 'Wings':False, 'Yellow':False, 'Blue':True},
    {'Four Wheels':True, 'Transports People':True, 'Wings':False, 'Yellow':True, 'Blue':False},
    {'Four Wheels':True, 'Transports People':True, 'Wings':False, 'Yellow':True, 'Blue':False}
]

## Creating Patterns with AND and NOT

The Tsetlin machine learns to recognize different object types by creating _if-then rules_. Each rule has the form: __if__ _condition_ __then__ _class_.

The _condition_ is an __and__-expression while the _class_ is a type of object. You can use the method `evaluate_condition()` to evaluate the condition of a rule:

In [2]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if 'NOT ' + feature in condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition

For instance, create the condition

In [3]:
example_condition = ['Four Wheels', 'Transports People', 'NOT Wings']

This condition means: _Four Wheels_ __and__ _Transports People_ __and__ __not__ _Wings_. Now, evaluate the condition on the first object in the dataset:

In [4]:
evaluate_condition(cars[0], example_condition)

True

Since, the first object in the dataset has _Four Wheels_, _Transports People_, and does not have _Wings_, the __and__-expression becomes _True_.

## Learning Frequent Patterns with Type I Feedback

### Rule Memory

You are now about to discover how a single if-then rule learns to recognizing frequent _Car_ patterns. To this end, the rule has its own memory for storing literals, visualized in the figure below.

![Alt text](https://github.com/olegranmo/blob/blob/c5dea3cf340914963c2cdcfe55c9f92ae04d6cec/Tsetlin_Machine_Memory_1_1.png?raw=true)

The y-axis ranges from 1 to 10 and measures how deeply the memory stores each literal:

* Values 1 to 5 stand for _Forgotten_. Value 1 means maximally forgotten while value 5 means almost memorized. Literals in this range _do not_ take part in the rule's condition.
* Values 6 to 10 mean _Memorized_. Value 6 stands for lightly retained, and value 10 represents maximally memorized. Literals in this range take part in the condition of the rule.

The memory in the figure is:

In [5]:
memory = {'Four Wheels':10, 'NOT Four Wheels':2, 'Transports People':9, 'NOT Transports People':3, 'Wings':1, 'NOT Wings':5, 'Yellow':1, 'NOT Yellow':4, 'Blue':6, 'NOT Blue':4}

The `get_condition()` method extracts the literals in memory position 6 to 10:

In [6]:
def get_condition():
    condition = []
    for literal in memory:
        if memory[literal] >= 6:
            condition.append(literal)
    return condition

You can try `get_condition()` on the above example memory:

In [7]:
get_condition()

['Four Wheels', 'Transports People', 'Blue']

The condition is _Four Wheels_ __and__ _Transports People_ __and__ _Blue_.

### Initialization

When learning begins the rule starts up with all the literals in memory position 5. That is, the literals are about to be _Memorized_ but currently _Forgotten_.

In [8]:
memory = {'Four Wheels':5, 'NOT Four Wheels':5, 'Transports People':5, 'NOT Transports People':5, 'Wings':5, 'NOT Wings':5, 'Yellow':5, 'NOT Yellow':5, 'Blue':5, 'NOT Blue':5}

### Randomization

Learning becomes flexible through randomization. So, you will need a couple of methods from the Python `random` module.

In [9]:
from random import random
from random import choice

### Memorization and Forgetting

The _memorize value_ 0.1 decides how quickly the rule memorizes literals. Conversely, the _forget value_ 0.9 decides how quickly the rule forgets in the absence of observations.

In [10]:
memorize_value = 0.1
forget_value = 0.9

You memorize a literal by incrementing its position in memory unless already _Maximally Memorized_ in position 10. The memorization thus pushes the literals deeper into the rule's memory. The `memorize()` method implements memorization:

In [11]:
def memorize(literal):
    if random() <= memorize_value and memory[literal] < 10:
        memory[literal] += 1

You forget a literal by decrementing its position in memory unless already _Maximally Forgotten_ in position 1. The `forget()` method implements forgetting:

In [12]:
def forget(literal):
    if random() <= forget_value and memory[literal] > 1:
        memory[literal] -= 1

Try out forget two times on _Yellow_ and observe how the memory updates:

In [13]:
forget('Yellow')
forget('Yellow')
print(memory)

{'Four Wheels': 5, 'NOT Four Wheels': 5, 'Transports People': 5, 'NOT Transports People': 5, 'Wings': 5, 'NOT Wings': 5, 'Yellow': 3, 'NOT Yellow': 5, 'Blue': 5, 'NOT Blue': 5}


_Yellow_ is now either in position 3, 4, or 5 depending on how many times `random` was equal to or below 0.9.

### Type I Feedback

The `type_i_feedback()` method produces frequent patterns with two learning steps:

1. Check if the condition part of the rule is _True_ by assessing the object's literals. If the condition part is _True_, then memorize all the literals that are _True_ for the object. You memorize a literal by incrementing its position in memory unless already _Maximally Memorized_ in position 10. The memorization thus pushes the literals deeper into the rule's memory.

2. Forget all remaining literals by pushing them towards being _Maximally Forgotten_. You forget a literal by decrementing its position in memory unless already _Maximally Forgotten_ in position 1.

In [14]:
def type_i_feedback(observation):
    remaining_literals = list(memory.keys())
    if evaluate_condition(observation, get_condition()) == True:
        for feature in observation:
            if observation[feature] == True:
                memorize(feature)
                remaining_literals.remove(feature)
            elif observation[feature] == False:
                memorize('NOT ' + feature)
                remaining_literals.remove('NOT ' + feature)
    for literal in remaining_literals:
        forget(literal)

### Example Run

The following code randomly selects a car and then updates the rule. This procedure is repeated 100 times.

In [15]:
for i in range(100):
    observation_id = choice([0,1,2])
    type_i_feedback(cars[observation_id])

The memory now looks like this:

In [16]:
print(memory)

{'Four Wheels': 10, 'NOT Four Wheels': 1, 'Transports People': 8, 'NOT Transports People': 1, 'Wings': 1, 'NOT Wings': 10, 'Yellow': 1, 'NOT Yellow': 1, 'Blue': 1, 'NOT Blue': 1}


Observe that some of the literals are now either deeply memorized or forgotten. You can print out the resulting rule:

In [17]:
print("IF " + " AND ".join(get_condition()) + " THEN Car")

IF Four Wheels AND Transports People AND NOT Wings THEN Car


## Increasing Discrimination Power withType II Feedback

You can use a dataset with planes to make the rule for recognizing cars more discriminative:

In [18]:
planes = [
    {'Four Wheels':True, 'Transports People':True, 'Wings':True, 'Yellow':False, 'Blue':True},
    {'Four Wheels':True, 'Transports People':False, 'Wings':True, 'Yellow':True, 'Blue':False},
    {'Four Wheels':False, 'Transports People':True, 'Wings':True, 'Yellow':False, 'Blue':True}
]

### Type II Feedback

The `type_ii_feedback()` method implements the third and final learning step:

3. Check if the condition part of the rule is _True_ by assessing the object’s literals. If the condition part  is _True_, then  memorize  all _Forgotten_ literals that are _False_ for the object.  Again, you memorize a literal by incrementing its position in memory.  However, this time there is no randomization – the increment is always performed. In effect, the memorization pushes the literals that are in memory position 1 to 5, and at the same time are _False_, towards being _Memorized_.

In [19]:
def memorize_always(literal):
    memory[literal] += 1

In [20]:
def type_ii_feedback(observation):
    if evaluate_condition(observation, get_condition()) == True:
        for feature in observation:
            if observation[feature] == False:
                memorize_always(feature)
            elif observation[feature] == True:
                memorize_always('NOT ' + feature)

### Example Run

Our example rule

Let us assume that we have a less discriminative rule:

In [21]:
memory = {'Four Wheels': 10, 'NOT Four Wheels': 1, 'Transports People': 10, 'NOT Transports People': 1, 'Wings': 1, 'NOT Wings': 1, 'Yellow': 1, 'NOT Yellow': 1, 'Blue': 1, 'NOT Blue': 1}
print("IF " + " AND ".join(get_condition()) + " THEN Car")

IF Four Wheels AND Transports People THEN Car


The rule correctly matches all of the cars in the _Car_ dataset. However, it also erroneously matches the first object in the _Plane_ dataset:

In [22]:
print(evaluate_condition(planes[0], get_condition()))

True


A few rounds of Type II Feedback fix this error:

In [23]:
for i in range(100):
    observation_id = choice([0,1,2])
    type_ii_feedback(planes[observation_id])
print(memory)

{'Four Wheels': 10, 'NOT Four Wheels': 6, 'Transports People': 10, 'NOT Transports People': 6, 'Wings': 1, 'NOT Wings': 6, 'Yellow': 6, 'NOT Yellow': 1, 'Blue': 1, 'NOT Blue': 6}


Observe how the feedback has pushed several _False_ literals to memory position 6. The rule no longer recognizes the offending plane:

In [24]:
print("IF " + " AND ".join(get_condition()) + " THEN Car")
print(evaluate_condition(planes[0], get_condition()))

IF Four Wheels AND NOT Four Wheels AND Transports People AND NOT Transports People AND NOT Wings AND Yellow AND NOT Blue THEN Car
False


Unfortunately, the rule is no longer frequent for cars either. The key strategy is to mix Type I and Type II Feedback. Then the rule becomes both frequent and discriminative:

In [25]:
for i in range(100):
    observation_id = choice([0,1,2])
    car = choice([0,1])
    if car == 1:
        type_i_feedback(cars[observation_id])
    else:
        type_ii_feedback(planes[observation_id])

In [26]:
print(memory)
print("IF " + " AND ".join(get_condition()) + " THEN Car")

{'Four Wheels': 10, 'NOT Four Wheels': 1, 'Transports People': 9, 'NOT Transports People': 1, 'Wings': 1, 'NOT Wings': 8, 'Yellow': 2, 'NOT Yellow': 1, 'Blue': 1, 'NOT Blue': 1}
IF Four Wheels AND Transports People AND NOT Wings THEN Car
